In [14]:
!pip install --upgrade pip

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
import seaborn as sns

# from scipy.special import entr
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer, MaxAbsScaler, RobustScaler, PowerTransformer

In [15]:
CORES = -1
SEED = 567

In [16]:
MAGENTA = (202/255, 18/255, 125/255)

In [17]:
get_new_scaler_dict = {"StandardScaler": StandardScaler, "MinMaxScaler": MinMaxScaler, "Normalizer": Normalizer, 
                       "MaxAbsScaler": MaxAbsScaler, "RobustScaler": RobustScaler, "PowerTransformer": PowerTransformer}

sns.set(rc={'figure.figsize':(12,8), 'figure.dpi': 180, "legend.fontsize": 26, "axes.labelsize": 26, #"xtick.fontsize": 26, "ytick.fontsize": 20, 
            "xtick.labelsize": 20, "ytick.labelsize": 20})

err_distro_rc={'figure.figsize':(20,10), 'figure.dpi': 180, "legend.fontsize": 30, "axes.labelsize": 30, #"xtick.fontsize": 26, "ytick.fontsize": 20, 
            "xtick.labelsize": 25, "ytick.labelsize": 25}

In [18]:
sns.set(rc={'figure.figsize':(12,8), 'figure.dpi': 180, "legend.fontsize": 9, "axes.labelsize": 11, #"xtick.fontsize": 26, "ytick.fontsize": 20, 
            "xtick.labelsize": 11, "ytick.labelsize": 11})
# sns.set(rc={'figure.figsize':(12,8), 'figure.dpi': 180, "legend.fontsize":11, "axes.labelsize": 11, #"xtick.fontsize": 26, "ytick.fontsize": 20, 
#             "xtick.labelsize": 11, "ytick.labelsize": 11})

sns.set_style("darkgrid")
sns.set_context("paper")
sns.set(font="sans")
sns.set_palette("tab10")

# for plain matplotlib:
plt.style.use(["seaborn-darkgrid", "seaborn-paper"])
plt.rc("font", family="sans", size=8)
plt.rc("axes", titlesize=6)
plt.rc("axes", labelsize=6)
plt.rc("xtick", labelsize=6)
plt.rc("ytick", labelsize=6)
plt.rc("xtick.major", pad=1)
plt.rc("ytick.major", pad=3)
plt.rc("legend", fontsize=6)
plt.rc("figure", titlesize=6)

In [19]:
# Define a style I use a lot for boxplots:
utils_boxplot_style = dict(
    color="tab:blue",
    linewidth=0.5,
    saturation=1,
    width=0.7,
    flierprops=dict(
        marker="o", markersize=2, markerfacecolor="none", markeredgewidth=0.5
    ),
)

# Define a style I use a lot for lineplots:
utils_lineplot_style = dict(
    color="tab:blue", linewidth=0.5, marker="o", markersize=3, markeredgewidth=0.5
)

In [20]:
print("\x1b[32m\"SEED and CONSTANTS imported\"\x1b[0m")

"SEED and CONSTANTS imported"
